# Running drivers

Each driver can be run from **bash** or **Python**. Only one example below for `chgres_cube` is shown using the Python interface (API).

In [ ]:
# Set up the environment using Python so it's available to each bash subshell below, along with all the Python cells.

import os
from datetime import datetime, timedelta
from pathlib import Path
uw_training = "/scratch3/BMC/wrfruc/cholt/uw_training_prep/uw_training"
os.environ["uw_training"] = uw_training

cycle = datetime(2025, 8, 27, 12)

## Run chgres_cube for Initial Conditions

In [ ]:
%%script bash -l
source /etc/profile.d/slurm.sh
uw chgres_cube run --cycle 2025-08-27T12 --leadtime 0 -c $uw_training/uw_configs/prepare_bcs.yaml --batch --key-path make_ics

### Python API Alternative for chgres_cube

Docs: __[uwtools.api.chgres_cube](https://uwtools.readthedocs.io/en/main/sections/user_guide/api/chgres_cube.html)__


In [ ]:
from uwtools.api.chgres_cube import ChgresCube
from uwtools.api.logging import use_uwtools_logger

use_uwtools_logger()

ics = ChgresCube(config=Path(uw_training, "uw_configs", "prepare_bcs.yaml") , cycle=cycle, leadtime=timedelta(hours=0), key_path=["make_ics"])
ics.provisioned_rundir()

In [ ]:
help(ics)

In [ ]:
ics.config

In [ ]:
ics.output

## Run chgres_cube for Lateral Boundary Conditions

Run once for each lead time in a loop. Always point to the same section of the UW YAML.

In [ ]:
%%script bash -l
for i in $(seq 0 6) ; do uw chgres_cube run --cycle 2025-08-27T12 --leadtime $i -c $uw_training/uw_configs/prepare_bcs.yaml --batch --key-path make_lbcs ; done

## Run a Cold-start FV3 forecast

In [ ]:
%%script bash -l
uw fv3 run --cycle 2025-08-27T12 -c $uw_training/uw_configs/fv3_config.yaml --batch --key-path forecast_prod

## Run the Analysis at the *NEXT* Cycle

Here, the cycling frequency is 1 hour.

In [ ]:
%%script bash -l
# Run with -q flag because the notebook can't handle the output in a timely fashion. Subsequent runs after the links are made can remove it.
uw gsi run --cycle 2025-08-27T13 -c $uw_training/uw_configs/gsi_uw_config.yaml --key-path analysis --batch -q

## Run a Cycled FV3 forecast from the Updated Analysis 

* This forecast was configured to run using the lateral boundary conditions from the previous cycle.
* It runs a script that removes the checksums from the analysis-update files between staging the files and submitting the job.

In [ ]:
%%bash
cat $uw_training/scripts/run_forecast.sh

In [ ]:
%%script bash -l
$uw_training/scripts/run_forecast.sh $uw_training/uw_configs/fv3_config.yaml 2025-08-27T13